# Empirical Strategy 
I estimate the following equation
\begin{equation} y_{i} = \beta_0 + \beta_1 x_{i} + \beta_2\text{controls}_{i} + \varepsilon_{i} \label{main-eq}\tag{1} \end{equation}

where $y_i$ represents cumulative utility-scale solar installations in a given year (2016 or 2019). 

Variables $x_i$ include market design characteristics that may affect the supply of solar:

- Solar carve-out (% target in 2016)
- Renewable portfolio standard (% target in 2025)
- Whether a state offered a large-scale subsidy between 2010-2016 (=1 for OR, NC, AZ, and NM)
- Whether a state allows retail choice
- Whether a state belongs to an ISO or RTO 

Controls include
- Population
- State GDP
- Area (in acres)
- Solar resource (capacity facotr) 

In [14]:
**********************************************
****
**** OPENING DATA FOR Rachel Anderson
****
**********************************************
set more off
clear all
macro drop _all
clear matrix
drop _all 
cap log close
estimates clear

import delimited "/Users/rachelanderson/Dropbox (Princeton)/Research_V2/descriptive_solar/code/analysis/stata-policy-analysis/state_policy_panel.csv"

**********************************************
****
**** Encode group identifiers
****
**********************************************

egen state_code = group(state)
egen region_code = group(region)

**********************************************
****
**** Outcomes
****
**********************************************


// Create log outcome variables
gen log_n_plants = log(n_plants+1)
gen log_ac_cap = log(ac_cap_added + 1)

gen log_n_plants_qf = log(n_plants_qf + 1)
gen log_ac_cap_qf = log(ac_cap_added_qf + 1)

gen log_cum_ac_cap = log(cum_ac_cap_added + 1)
gen log_cum_n_plants = log(cum_n_plants + 1)

gen cum_avg_cap = cum_ac_cap_added/cum_n_plants
replace cum_avg_cap = 0 if cum_avg_cap == .
replace avg_cap_ac = 0 if avg_cap_ac == .

gen log_cum_avg_plant = log(cum_avg_cap + 1)
label variable log_cum_avg_plant "Log Avg. Size"

label variable log_cum_ac_cap "Log Total Capacity"
label variable log_ac_cap "Log Capacity Additions"

label variable log_n_plants "Log # Plants Added"
label variable log_cum_n_plants "Log Total # Plants"

label variable cum_avg_cap "Avg. plant size"
label variable avg_cap_ac "Avg. new plant size"

gen log_avg_cap = log(avg_cap_ac + 1)
label variable log_avg_cap "Log avg. size"

// Create non-QF variables for robustness checks
global outcomes "n_plants ac_cap_added"
foreach i in $outcomes{
    gen `i'_non_qf = `i' - `i'_qf
    gen log_`i'_non_qf = log(`i'_non_qf + 1)
}

**********************************************
****
**** Controls
****
**********************************************

gen log_acres = log(acres)
gen log_cf = log(nrel_cap_factor)

global controls "log_cf log_acres log_pop retail_choice log_gdp"
global retail_controls "p_public_sales_2016 p_retail_sales_2016"

label variable log_cf "Log Capacity Factor"
label variable nrel_cap_factor "Capacity Factor"

label variable acres "Acres"
label variable log_acres "Log acres"

label variable p_retail_sales_2016 "% Retail Sales (2016)"
label variable p_iou_sales_2016 "% IOU Sales (2016)"
label variable retail_choice "Retail Choice?"
label variable iso_rto_dummy "ISO/RTO?"

label variable log_pop "Log population"
label variable log_sales "Log retail sales"

gen log_gdp = log(gdp)
label variable log_gdp "Log GDP"

**********************************************
****
**** Policies
****
**********************************************

// PURPA Characteristics 

global purpa "purpa_wholesale_2020 purpa_competitive_2020 purpa_standard_contract_2020 purpa_mw_limit_2020 purpa_max_term_2020"

gen purpa_long_term = 0 
replace purpa_long_term = 1 if purpa_max_term_2020 > 0 

gen purpa_large_limit = 0
replace purpa_large_limit = 1 if purpa_mw_limit_2020 >= 1

gen purpa_long_term_large = purpa_long_term * purpa_large_limit 

global alt_purpa "purpa_long_term purpa_large_limit purpa_long_term_large"

label variable purpa_wholesale_2020 "PURPA Wholesale Prices"
label variable purpa_competitive_2020 "Competitive PURPA"
label variable purpa_standard_contract_2020 "Standard Offer"
label variable purpa_mw_limit_2020 "Max PURPA Size"
label variable purpa_max_term_2020 "Max PURPA Contract Length"

label variable purpa_large_limit "PURPA Cap > 1 MW"
label variable purpa_long_term "Long-term PURPA"
label variable purpa_long_term_large "Contracts for Large QFs"


// ITC/PTC Characteristics  

global itc_ptc "itc_dummy ptc_dummy"
global alt_itc_ptc "itc_dummy alt_ptc_dummy"

label variable itc_dummy "State ITC"
label variable alt_ptc_dummy "State PTC"

gen any_sub = 0
replace any_sub = 1 if state == "NC" | state == "OR" | state == "AZ" | state == "NM"
label variable any_sub "Large-scale subsidy"

//  RPS Targets  

global rps_2016 "tot_rps_lbl_target_2016 solar_rps_manual_target_2016 dg_rps_manual_target_2016"
global rps_2025 "tot_rps_lbl_target_2025 solar_rps_manual_target_2025 dg_rps_manual_target_2025"

label variable tot_rps_lbl_target_2016 "RPS % Target (2016)"
label variable tot_rps_lbl_target_2025 "RPS % Target (2025)"
label variable tot_rps_lbl_mwh_2016 "RPS MWh Target (2016)"
label variable tot_rps_lbl_mwh_2025 "RPS MWh Target (2025)"

label variable solar_rps_manual_target_2016 "Solar % Target (2016)"
label variable solar_rps_manual_target_2025 "Solar % Target (2025)"
label variable dg_rps_manual_target_2016 "DG % Target (2016)"
label variable dg_rps_manual_target_2025 "DG % Target (2025)"

// RPS and Retail Choice Interaction

gen retail_solar_rps = retail_choice * solar_rps_manual_target_2016
gen retail_dg_rps = retail_choice * dg_rps_manual_target_2016
gen retail_tot_rps = retail_choice* tot_rps_lbl_target_2025
label variable retail_solar_rps "Retail Choice x Solar RPS"
label variable retail_dg_rps "Retail Choice x DG RPS"
label variable retail_tot_rps "Retail Choice x RPS"

// Any solar RPS dummy (for split sample, robustness)

gen any_solar_rps_2016 = 0 
replace any_solar_rps_2016 = 1 if solar_rps_manual_target_2016 > 0 
replace any_solar_rps_2016 = 1 if dg_rps_manual_mwh_2016 > 0 
label variable any_solar_rps_2016 "Solar Carve-out (2016?)"

gen any_solar_rps_2025 = 0 
replace any_solar_rps_2025 = 1 if solar_rps_manual_target_2025 > 0 
replace any_solar_rps_2025 = 1 if dg_rps_manual_target_2025 > 0 
label variable any_solar_rps_2025 "Solar Carve-out (2025)?"

gen log_solar_rps_mwh_2016 = log(solar_rps_manual_mwh_2016 + 1)
label variable log_solar_rps_mwh_2016 "Log MWh Target (2016)"

gen log_solar_rps_mwh_2025 = log(solar_rps_manual_mwh_2025 + 1)
label variable log_solar_rps_mwh_2025 "Log MWh Target (2025)"









(60 vars, 500 obs)









(172 missing values generated)

(172 real changes made)

(0 real changes made)































(120 real changes made)


(280 real changes made)
















(40 real changes made)



















(140 real changes made)

(10 real changes made)



(140 real changes made)

(10 real changes made)







In [15]:
// Results are robust to excluding California

global controls "log_cf log_acres log_pop log_gdp retail_choice iso_rto_dummy p_iou_sales_2016"
global rps_vars "solar_rps_manual_target_2016 dg_rps_manual_target_2016 tot_rps_lbl_target_2025"
global purpa_vars "purpa_competitive_2020 purpa_long_term_large"

estimates clear
eststo: quietly reg log_cum_ac_cap $rps_vars $purpa_vars any_sub $controls if year == 2016, r
eststo: quietly reg log_cum_n_plants $rps_vars $purpa_vars any_sub $controls if year == 2016, r cluster(state_code)
eststo: quietly reg log_cum_avg_plant $rps_vars $purpa_vars any_sub $controls if year == 2016, r cluster(state_code)
esttab, label noabbrev mtitle("Log cap" "Log # Plants" "Log Avg. Size") nonumber title(Full Sample Results in 2016)






(est1 stored)

(est2 stored)

(est3 stored)


Full Sample Results in 2016
--------------------------------------------------------------------
                          Log cap    Log # Plants    Log Avg. Size   
--------------------------------------------------------------------
Solar % Target (2016)        0.750*          0.634***       0.0949   
                           (2.71)          (4.50)          (0.53)   

DG % Target (2016)          0.765           0.558          0.0442   
                           (1.21)          (1.66)          (0.12)   

RPS % Target (2025)        0.0256          0.0214         0.00220   
                           (1.11)          (1.57)          (0.19)   

Competitive PURPA           0.847           0.496           0.172   
                           (0.79)          (0.95)          (0.28)   

Contracts for Large QFs        0.687           0.943*         -0.470   
                           (0.73)          (2.18)         (-0.82)   

Large-scale su

In [16]:
// Results are robust to excluding California

global controls "log_cf log_acres log_pop log_gdp retail_choice iso_rto_dummy p_iou_sales_2016"
global rps_vars "solar_rps_manual_target_2016 dg_rps_manual_target_2016 tot_rps_lbl_target_2025"
global purpa_vars "purpa_competitive_2020 purpa_long_term_large"

estimates clear
eststo: quietly reg log_cum_ac_cap $rps_vars $purpa_vars any_sub $controls if year == 2019, r
eststo: quietly reg log_cum_n_plants $rps_vars $purpa_vars any_sub $controls if year == 2019, r cluster(state_code)
eststo: quietly reg log_cum_avg_plant $rps_vars $purpa_vars any_sub $controls if year == 2019, r cluster(state_code)
esttab, label noabbrev mtitle("Log cap" "Log # Plants" "Log Avg. Size") nonumber title(Full Sample Results in 2016)






(est1 stored)

(est2 stored)

(est3 stored)


Full Sample Results in 2016
--------------------------------------------------------------------
                          Log cap    Log # Plants    Log Avg. Size   
--------------------------------------------------------------------
Solar % Target (2016)        0.624           0.586***       0.0351   
                           (1.97)          (3.67)          (0.18)   

DG % Target (2016)          0.456           0.318          0.0579   
                           (0.69)          (1.00)          (0.12)   

RPS % Target (2025)        0.0234          0.0233        -0.00272   
                           (0.99)          (1.64)         (-0.23)   

Competitive PURPA           0.703           0.259           0.246   
                           (0.90)          (0.55)          (0.58)   

Contracts for Large QFs        1.484           1.137**         0.323   
                           (1.99)          (3.01)          (0.61)   

Large-scale su